In [39]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [41]:
from google.colab import files

files.upload()

{}

In [42]:
! cp kaggle.json ~/.kaggle/

In [43]:
! chmod 600 ~/.kaggle/kaggle.json

In [44]:
! kaggle datasets download -d pavansubhasht/ibm-hr-analytics-attrition-dataset

ibm-hr-analytics-attrition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [45]:
import pandas as pd
import numpy as np

In [46]:
attr_data = pd.read_csv("/content/ibm-hr-analytics-attrition-dataset.zip")
attr_data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


The dataset:

*   has both numerical and categorical values
*   has possibly null or infinite values



In [47]:
print(attr_data.isnull().any())

Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesL

We start with separating Numerical and Categorical columns and use OrdinalEncoder to encode Categorical columns.

In [48]:
print("categorical cols:")
Categorical = [col for col, value in attr_data.iteritems() if value.dtype == 'object']
print(Categorical)
print("\nnumerical cols:")
Numerical = [col for col, value in attr_data.iteritems() if value.dtype != 'object']
print(Numerical)

categorical cols:
['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']

numerical cols:
['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


In [49]:
from sklearn.preprocessing import OrdinalEncoder
attr_data_cat = attr_data.copy()

ord_enc = OrdinalEncoder()
attr_data_cat[Categorical] = ord_enc.fit_transform(attr_data[Categorical])
print(attr_data_cat.head(2))

   Age  Attrition  BusinessTravel  DailyRate  Department  DistanceFromHome  \
0   41        1.0             2.0       1102         2.0                 1   
1   49        0.0             1.0        279         1.0                 8   

   Education  EducationField  EmployeeCount  EmployeeNumber  ...  \
0          2             1.0              1               1  ...   
1          1             1.0              1               2  ...   

   RelationshipSatisfaction  StandardHours  StockOptionLevel  \
0                         1             80                 0   
1                         4             80                 1   

   TotalWorkingYears  TrainingTimesLastYear  WorkLifeBalance  YearsAtCompany  \
0                  8                      0                1               6   
1                 10                      3                3              10   

   YearsInCurrentRole  YearsSinceLastPromotion  YearsWithCurrManager  
0                   4                        0         

Dropping the Target value [Attrition] column from the x set

In [50]:
attr_data_y = attr_data_cat[['Attrition']]
attr_data_x = attr_data_cat.drop("Attrition", axis=1)
print(attr_data_y.head(3))
print(attr_data_x.head(1))

   Attrition
0        1.0
1        0.0
2        1.0
   Age  BusinessTravel  DailyRate  Department  DistanceFromHome  Education  \
0   41             2.0       1102         2.0                 1          2   

   EducationField  EmployeeCount  EmployeeNumber  EnvironmentSatisfaction  \
0             1.0              1               1                        2   

   ...  RelationshipSatisfaction  StandardHours  StockOptionLevel  \
0  ...                         1             80                 0   

   TotalWorkingYears  TrainingTimesLastYear  WorkLifeBalance  YearsAtCompany  \
0                  8                      0                1               6   

   YearsInCurrentRole  YearsSinceLastPromotion  YearsWithCurrManager  
0                   4                        0                     5  

[1 rows x 34 columns]


Logistic Regression to predict attrition

In [51]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(attr_data_x, attr_data_y, train_size= 0.80, random_state=1); 

In [52]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.fit_transform(test_x)

In [53]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg = logReg.fit(train_x, train_y)
print('Accuracy =', logReg.score(train_x, train_y)*100,'%');

Accuracy = 88.35034013605441 %


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [54]:
from sklearn.metrics import accuracy_score
logReg_predictions = logReg.predict(test_x)
print('Test Accuracy Score:', accuracy_score(test_y, logReg_predictions)*100, '%\n')

Test Accuracy Score: 83.6734693877551 %

